# 0.1 Imports

In [2]:
import math
import pandas as pd
import numpy  as np


from statsmodels.stats import api as sms

# 0.2 Helper Functions


In [3]:
def jupyter_settings(): 
    %matplotlib inline 
    #%matplotlib notebook
    %pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
     
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option ('display.expand_frame_repr', False)
    
    sns.set()


# 1.0 Data Description

In [4]:
df_raw = pd.read_csv('/Users/adriele/Documents/repos/ab_testing/dataset/ab_data_james.csv')
df_raw.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [5]:
df_raw.isna().sum()

user_id         0
timestamp       0
group           0
landing_page    0
converted       0
dtype: int64

In [24]:
# Flags test A

df_user_delete = df_raw[['user_id', 'group']].groupby( ['user_id'] ).count().reset_index().query( 'group > 1') ['user_id']
# data without duplicates
df1 = df_raw[~df_raw['user_id'].isin( df_user_delete )]

# 2.0 Experiment Design

## 2.2 Experiment Paramiter

In [25]:
# A/B Test - paramiter
# A/B test - Permutacao
# Z-Test ( )


In [30]:
# confidence level
confidence_level = 0.95

# Significance Level
significance_level = 0.05

# Convertions new and old page
p1 = 0.13
p2 = 0.15

# effect size
effect_size = sms.proportion_effectsize( p1, p2)

# Statisc Power
power = 0.80

# Sample size
sample_n = sms.NormalIndPower().solve_power(
                effect_size,
                power=power,
                alpha=significance_level
)

sample_n = math.ceil( sample_n )
sample_n

4720

In [31]:
## 2.2 amostragem aleatoria
df_control_sample = df1[df1['group'] == 'control'].sample( n=sample_n, random_state=42 )
print ( 'Size of Control Group: {}'.format( df_control_sample.shape[0] ) )

# Treatment Group
df_treatment_sample = df1[df1['group'] == 'treatment'].sample( n=sample_n, random_state=42 )
print ( 'Size of Treatment Group: {}'.format( df_treatment_sample.shape[0] ) )

# Total Sample
df_ab = pd.concat([ df_control_sample, df_treatment_sample] ).reset_index( drop=True )

Size of Control Group: 4720
Size of Treatment Group: 4720


In [37]:
# metrics between groups
# ======================== Group Control =========================
sales = df_control_sample.loc[df_control_sample['converted'] == 1, 'converted'].sum()
visit = len( df_control_sample )

conversion_rate_control = sales / visit
print ( 'Convertion Rate - Control Group: {}'.format( conversion_rate_control ))

# ======================== Group Control =========================
sales = df_treatment_sample.loc[df_treatment_sample['converted'] == 1, 'converted'].sum()
visit = len( df_treatment_sample )

conversion_rate_treatment = sales / visit
print ( 'Convertion Rate - Treatment Group: {}'.format( conversion_rate_treatment ))


# conversao = compradores/ visitantes


Convertion Rate - Control Group: 0.11546610169491525
Convertion Rate - Treatment Group: 0.12902542372881357
